<a href="https://colab.research.google.com/github/Sandeep-4469/Chexpert_solution/blob/main/DSP_Project_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install openai==0.28 gradio

In [6]:
import os
import zipfile
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import openai

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])




extracted_dir = "/content/drive/MyDrive/fine_tuned_models"
image_path = "/content/view1_frontal.jpg"

model_files = [f for f in os.listdir(extracted_dir) if f.endswith(".pth")]
diseases = [name.split("_")[0] for name in model_files]

image = Image.open(image_path).convert("RGB")
image = valid_transforms(image).unsqueeze(0).to(device)

probabilities = {}

for model_file, disease in zip(model_files, diseases):
    base_model = models.densenet121(pretrained=True)
    for param in list(base_model.parameters())[:-5]:
        param.requires_grad = False
    num_features = base_model.classifier.in_features
    base_model.classifier = nn.Sequential(
        nn.Linear(num_features, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 1),
        nn.Sigmoid()
    )

    model_path = os.path.join(extracted_dir, model_file)
    base_model.load_state_dict(torch.load(model_path, map_location=device))
    base_model.to(device)
    base_model.eval()

    with torch.no_grad():
        output = base_model(image)
        probability = output.item()
        probabilities[disease] = 1-probability

print("Predicted Probabilities:")
for disease, prob in probabilities.items():
    print(f"{disease}: {prob:.4f}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-6-1b4866c91db1>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more 

Predicted Probabilities:
Atelectasis: 0.2482
Cardiomegaly: 0.1521
Consolidation: 0.0296
Edema: 0.1291
Pleural: 0.0294
Pneumonia: 0.0992


In [7]:
openai.api_key = "API_KEY"

In [8]:
import gradio as gr
import openai

# Define the context for lung diseases
context = f"""
You are a doctorbot specializing in lung diseases.
The patient has provided an X-ray, and the analysis reveals the following probabilities of lung-related conditions:
{probabilities}.

Your tasks are:
1. ask for name,
2. ask for age after telling his/her name
3. then ask for any smoking habits after age
1. Greet the patient by name and present the X-ray findings clearly with probabilities.
2. Ask the user if they have any specific questions about the report or symptoms (e.g., difficulty breathing, coughing, fever, chest pain).
3. Inquire about smoking history, recent respiratory infections, allergies, or pollutant exposure for relevant conditions like Pleural Effusion, Pneumonia, etc.
4. Provide guidance on managing lung health and suggest consulting a pulmonologist for further evaluation.
5. If the user asks out-of-context questions, politely redirect the conversation to lung health topics.
6. Maintain a concise and professional tone, tailoring responses to the user's questions and medical history if shared.

Begin by greeting the user with their name and presenting the X-ray results.
"""

# Define the chatbot function
def chat(message, history):
    try:
        # Reconstruct the conversation history
        prompt = context + "\n\n"
        if history:
            for user_message, bot_response in history:
                prompt += f"User: {user_message}\nDoctorbot: {bot_response}\n"

        prompt += f"User: {message}\nDoctorbot:"

        # Call OpenAI API to get the response
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": context}, {"role": "user", "content": prompt}]
        )

        # Return the bot's response
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Launch the chatbot interface
iface = gr.ChatInterface(fn=chat)
iface.launch()


/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:231: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://163f6589a0c564225e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
